In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [ ]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [20]:
jobs = pd.read_csv('jobs.csv')
parcels = pd.read_csv('parcels.csv')
buildings = pd.read_csv('buildings.csv')

In [11]:
parcels.columns

Index(['parcel_id_REMM', 'CO_NAME', 'COUNTY_ID', 'PARCEL_ID',
       'TOTAL_MKT_VALUE', 'land_value', 'residential_units', 'building_sqft',
       'FLOORS_CNT', 'year_built', 'EFFBUILT_YR', 'IS_OUG', 'max_dua',
       'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5',
       'type6', 'type7', 'type8', 'agriculture', 'basebldg', 'NoBuild',
       'redev_friction', 'building_type_id', 'x', 'y', 'parcel_acres', 'Split',
       'Split_Factor', 'parcel_id_REMM_old', 'building_type',
       'residential_price', 'non_residential_price', 'non_residential_sqft',
       'TAZID_832', 'TAZID_900', 'Old_PID', 'parent_parcel', 'elevation',
       'fwy_exit', 'airport', 'rail_depot', 'stream', 'trail', 'university',
       'shape_area', 'volume_one_way', 'volume_two_way', 'airport_distance',
       'fwy_exit_dist', 'raildepot_dist', 'university_dist', 'trail_dist',
       'stream_dist', 'train_station', 'rail_stn_dist', 'bus_rte_dist',
       'bus_stop', 'bus_stop_dist', 'volume_tw

In [21]:
buildings.columns

Index(['building_id', 'building_sqft', 'building_type_id',
       'non_residential_sqft', 'note', 'parcel_id', 'residential_units',
       'stories', 'unit_price_non_residential', 'year_built',
       'res_price_per_sqft', 'job_spaces'],
      dtype='object')

In [22]:
job_spaces_per_building = buildings.groupby(['building_id'])[['job_spaces']].sum().reset_index()
job_spaces_per_building.columns = ['building_id', 'job_spaces']
job_spaces_per_building

,building_id,job_spaces
0,1,0.0
1,2,967.0
2,3,6.0
3,4,12.0
4,5,24.0
...,...,...
712589,766007,0.0
712590,766008,0.0
712591,766009,0.0
712592,766010,0.0


In [3]:
jobs.head()

,jobs_id,building_id,cid,sector_id
0,0,27383,35,1
1,1,27383,35,1
2,2,27383,35,1
3,3,27383,35,1
4,4,27383,35,1


In [13]:
jobs_per_building = jobs.groupby(['building_id'])[['jobs_id']].count().reset_index()
jobs_per_building.columns = ['building_id', 'jobs']
jobs_per_building

,building_id,jobs
0,2,709
1,3,6
2,4,3
3,5,21
4,6,2
...,...,...
29958,665123,51
29959,665168,1061
29960,665401,23
29961,665879,190


In [28]:
# parcel id == building_id = True
merged = parcels.merge(jobs_per_building, left_on='parcel_id_REMM', right_on='building_id', how='left')
del merged['building_id']
merged = merged.merge(job_spaces_per_building, left_on='parcel_id_REMM', right_on='building_id', how='left')
merged = merged[['parcel_id_REMM', 'building_id','TAZID_832','jobs', 'job_spaces']]

In [30]:
jobs_per_taz = merged.groupby(['TAZID_832'])[['jobs']].sum().reset_index()
jobs_per_taz.columns = ['TAZID_832', 'jobs']

job_spaces_per_taz = merged.groupby(['TAZID_832'])[['job_spaces']].sum().reset_index()
job_spaces_per_taz.columns = ['TAZID_832', 'jobs_spaces']

In [31]:
employment = job_spaces_per_taz.merge(jobs_per_taz, on='TAZID_832', how='outer')
employment.to_csv('taz832_employment.csv', index=False)